In [230]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression, Ridge,Lasso,ElasticNet
from sklearn.preprocessing import PolynomialFeatures
from sklearn.pipeline import make_pipeline
from sklearn.dummy import DummyRegressor
from sklearn.model_selection import GridSearchCV


In [231]:
from sklearn.metrics import SCORERS
SCORERS.keys()

dict_keys(['explained_variance', 'r2', 'max_error', 'neg_median_absolute_error', 'neg_mean_absolute_error', 'neg_mean_absolute_percentage_error', 'neg_mean_squared_error', 'neg_mean_squared_log_error', 'neg_root_mean_squared_error', 'neg_mean_poisson_deviance', 'neg_mean_gamma_deviance', 'accuracy', 'top_k_accuracy', 'roc_auc', 'roc_auc_ovr', 'roc_auc_ovo', 'roc_auc_ovr_weighted', 'roc_auc_ovo_weighted', 'balanced_accuracy', 'average_precision', 'neg_log_loss', 'neg_brier_score', 'adjusted_rand_score', 'rand_score', 'homogeneity_score', 'completeness_score', 'v_measure_score', 'mutual_info_score', 'adjusted_mutual_info_score', 'normalized_mutual_info_score', 'fowlkes_mallows_score', 'precision', 'precision_macro', 'precision_micro', 'precision_samples', 'precision_weighted', 'recall', 'recall_macro', 'recall_micro', 'recall_samples', 'recall_weighted', 'f1', 'f1_macro', 'f1_micro', 'f1_samples', 'f1_weighted', 'jaccard', 'jaccard_macro', 'jaccard_micro', 'jaccard_samples', 'jaccard_wei

In [232]:
X_train = pd.read_csv('csv/X_train.csv')
X_test = pd.read_csv('csv/X_test.csv')
y_train = pd.read_csv('csv/y_train.csv')
y_test = pd.read_csv('csv/y_test.csv') 

In [233]:

modelRid = make_pipeline( PolynomialFeatures(degree=2,interaction_only=False),Ridge(alpha=0.01))


modelRid.fit(X_train,y_train)

Pipeline(steps=[('polynomialfeatures', PolynomialFeatures()),
                ('ridge', Ridge(alpha=0.01))])

In [268]:
modelRid.score(X_test,y_test)

0.8497654450447315

In [250]:
prediction = modelRid.predict(X_test)
df_prediction = pd.DataFrame(prediction,columns=["prediction"])

In [251]:
df_prediction["resultat"] = y_test['price']

In [262]:
df_prediction["diff"]= ((df_prediction["resultat"] - df_prediction["prediction"])**2)**(1/2)

In [239]:
modelLasso = make_pipeline( PolynomialFeatures(2),Lasso(alpha=0.1))
modelLasso.fit(X_train,y_train)

c:\Users\Delval\anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:647: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.102e+14, tolerance: 2.385e+11
  model = cd_fast.enet_coordinate_descent(


Pipeline(steps=[('polynomialfeatures', PolynomialFeatures()),
                ('lasso', Lasso(alpha=0.1))])

In [240]:
modelLasso.score(X_test,y_test)

0.8497658323042154

In [241]:
prediction = modelLasso.predict(X_test)
prediction

array([ 734656.75738452,  603324.63381219,  506420.1758774 , ...,
        755750.69414896, 1301901.93028017, 1833111.2175508 ])

In [242]:
modelEN = make_pipeline( PolynomialFeatures(2),ElasticNet(alpha=0.1))
modelEN.fit(X_train,y_train)

c:\Users\Delval\anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:647: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.825e+14, tolerance: 2.385e+11
  model = cd_fast.enet_coordinate_descent(


Pipeline(steps=[('polynomialfeatures', PolynomialFeatures()),
                ('elasticnet', ElasticNet(alpha=0.1))])

In [243]:
modelEN.score(X_test,y_test)

0.8441207569012268

In [244]:
prediction = modelEN.predict(X_test)
prediction

array([ 736005.45595277,  595525.6004023 ,  502091.66626847, ...,
        744626.19301921, 1340262.87216059, 1821092.36284225])

In [245]:
modelLR = make_pipeline( PolynomialFeatures(2),LinearRegression())
modelLR.fit(X_train,y_train)

Pipeline(steps=[('polynomialfeatures', PolynomialFeatures()),
                ('linearregression', LinearRegression())])

In [246]:
modelLR.score(X_test,y_test)

0.8497611778505407

In [247]:
prediction = modelLR.predict(X_test)
prediction

array([[ 734462.   ],
       [ 603133.875],
       [ 506223.   ],
       ...,
       [ 755576.125],
       [1301753.75 ],
       [1832911.75 ]])

In [248]:
modelDummy = make_pipeline( PolynomialFeatures(2),DummyRegressor())
modelDummy.fit(X_train,y_train)
modelDummy.score(X_test,y_test)

-0.0009614766305579003

In [249]:
prediction = modelDummy.predict(X_test)
prediction

array([542361.76707571, 542361.76707571, 542361.76707571, ...,
       542361.76707571, 542361.76707571, 542361.76707571])